<a href="https://colab.research.google.com/github/HariTarz/Hand_sign_detection_using_tensorflow/blob/main/Hand_sign_detection_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b>Preparation of dataset</b>

## <b>1.Collecting realtime images for training and testing</b>

### <b>Import dependencies</b>

In [1]:
# !pip install opencv-python

In [2]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

### <b>Setup folders for images</b>

In [4]:
BASE_PATH = os.path.join('Object_detection', 'Tensorslow')
IMAGES_PATH = os.path.join(BASE_PATH, 'workspace', 'images')
print(IMAGES_PATH)

Object_detection/Tensorslow/workspace/images


In [10]:
if not os.path.exists(IMAGES_PATH):
  if os.name == 'posix': #For Linux
    !mkdir -p {IMAGES_PATH+'/train'} {IMAGES_PATH+'/test'}
  if os.name == 'nt': # For Windows
    !mkdir {IMAGES_PATH+'/train'} {IMAGES_PATH+'/test'}

### <b>Capturing images for different labels</b>

In [ ]:
labels = ['peace', 'thumbsup', 'thumbsdown', 'thankyou', 'livelong', 'vanakkam']

num_img_for_train = 7
num_img_for_train 3
total_images = num_img_for_train + num_img_for_train

In [ ]:
for label in labels:
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
      print("Can't open camera capture")

    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(total_images):
        print('Capturing image - {}'.format(imgnum))
        ret, frame = cap.read()

        if imgnum < num_img_for_train:
          folder = 'train'
        else:
          folder = 'test'

        imgname = os.path.join(IMAGES_PATH, folder, label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

### <b>Labeling images</b>

In [ ]:
# Installing labeling dependencies
# !pip install --upgrade pyqt5 lxml

In [ ]:
LABELIMG_PACKAGE_PATH = os.path.join(BASE_PATH, 'labeling')

In [ ]:
if not os.path.exists(LABELIMG_PACKAGE_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [ ]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
# Command to open the labeling package 
!cd {LABELIMG_PATH} && python labelImg.py

### <b>Compress the train and test folders for google colab (optional)</b>

In [ ]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

In [ ]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}

## <b>2.Training and Detection</b>

### <b>Setup paths</b>

In [ ]:
# Setting up pre-trained model path and link
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'          # Name in models zoo: SSD MobileNet V2 FPNLite 320x320
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

# Link for tensorflow models zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
# Import Operating System
import os

In [ ]:
# Paths for different directories
BASE_PATH = os.path.join('Object_detection', 'Tensorslow') # Should be same as the previous BASE_PATH 
paths = {
    'WORKSPACE_PATH': os.path.join(BASE_PATH, 'workspace'),
    'SCRIPTS_PATH': os.path.join(BASE_PATH,'scripts'),
    'APIMODEL_PATH': os.path.join(BASE_PATH,'models'),
    'ANNOTATION_PATH': os.path.join(BASE_PATH, 'workspace','annotations'),
    'IMAGE_PATH': os.path.join(BASE_PATH, 'workspace','images'),
    'MODEL_PATH': os.path.join(BASE_PATH, 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join(BASE_PATH, 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(BASE_PATH, 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(BASE_PATH, 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join(BASE_PATH, 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(BASE_PATH, 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join(BASE_PATH,'protoc')
 }

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
# TFrecord file generater name
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

# Image-label match '.pbtext' file name
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
# Some important file paths 
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

### <b>Install tenforflow object detection API and all their dependencies</b>

In [ ]:
# !pip install wget
import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 